In [1]:
import pandas as pd
import os
from nltk.tokenize import word_tokenize # For tokenization
from nltk.corpus import stopwords # For stopwords removal
from nltk.stem.wordnet import WordNetLemmatizer # For noise removal
from gensim import corpora
import gensim
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as sia
import datetime

In [2]:
data = pd.read_csv('final.csv')

C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,6,33,34,35,106,107) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

REVIEW_ID           MEMBER_ID                        NPSCLINIC__C  \
0  a2p1U000000RoK0QAK  0011U00000riNolQAE        Arvada City of Arvada Clinic   
1  a2p3g000000LAe2AAG  0011U00000riuzfQAA  Seven Hills Lombardo Center Clinic   
2  a2p3g000000LAenAAG  0011U00000rieIYQAY           Monroe Campus Park Clinic   
3  a2p1U000000l8FEQAY  0011U00000rjZH9QAM       Akron-White Pond Drive Clinic   
4  a2p1U000000Rv9VQAS  0011U00000rjesHQAQ      Euclid Lincoln Electric Clinic   

  APPOINTMENTPROVIDER__C SURVEYDATE__C SURVEY_MONTH SURVEY_YEAR  \
0                    NaN    07-09-2018       Jul-18        2018   
1     0031U00000olN5ZQAU    07-01-2020       Jul-20        2020   
2     0031U00000olN7aQAE     6/27/2020       Jun-20        2020   
3     0031U00000olN71QAE    10/25/2019       Oct-19        2019   
4     0031U00000olN8YQAU     9/24/2019       Sep-19        2019   

   SURVEYNUMBER__C  NPS                                  REASONNPSSCORE__C  \
0            20536   10                                                NaN   
1      11752005028    5                                                NaN   
2      11737091171   10                                                NaN   
3            31790   10                                                NaN   
4            31283    8  I like the facility and the doctor.  I felt co...   

   ... positive_score negative_score sentiment sentiment_label  nps_label  \
0  ...          0.677          0.323         1        Positive   Promoter   
1  ...          0.000          1.000         0         Neutral  Detractor   
2  ...          0.000          1.000         0         Neutral   Promoter   
3  ...          0.000          1.000         0         Neutral   Promoter   
4  ...          0.169          0.831         1        Positive    Passive   

                          CLIENT NAME                  PARENT CLIENT NAME  \
0                   State of Colorado                   State of Colorado   
1  Cleveland Bakers & Teamsters (CBT)  Cleveland Bakers & Teamsters (CBT)   
2                        Union County                        Union County   
3  Cleveland Bakers & Teamsters (CBT)  Cleveland Bakers & Teamsters (CBT)   
4                    Lincoln Electric                    Lincoln Electric   

     PARENT_CLIENT_ID MEMBER_DOB MEMBER_STREET  
0  0011U00000rhpLpQAI        NaN           NaN  
1  0011U00000rhpN6QAI        NaN           NaN  
2  0011U00000rhpLjQAI        NaN           NaN  
3  0011U00000rhpN6QAI        NaN           NaN  
4  0011U00000rhpMwQAI        NaN           NaN  

[5 rows x 108 columns]

In [4]:
# Remove entries with no response and remove punctuations
data_reason_score = data[data['reviews'] != 'No Response']
data_reason_score = data_reason_score.reset_index(drop=True)
remove_punctuations_reason_score = data_reason_score[['reviews']]
remove_punctuations_reason_score.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
data_reason_score[['reviews']] = remove_punctuations_reason_score[['reviews']]

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [5]:
# Removing missing values
reason_score_reviews = data_reason_score
reason_score_reviews = reason_score_reviews.dropna(subset=['reviews'])
reason_score_reviews = reason_score_reviews.reset_index(drop=True)

In [6]:
# Converting reviews to lower case
reason_score_lower = reason_score_reviews[['reviews']]
reason_score_reviews['reviews'] = reason_score_lower.applymap(str.lower)

## Sentiment

In [7]:
# Finding polarity score for each feedback
# It will show the polarity scores (positive, negative, neutral, and compound) for each feedback along with the feedback received
# Polarity scores will then be inserted into a list which will be accessed later to compute sentiment scores
reason_score_polarity_score_list = []

for each in reason_score_reviews.reviews:
    polarity_score = sia().polarity_scores(each)
    polarity_score['label'] = each
    reason_score_polarity_score_list.append(polarity_score)

In [8]:
reason_score_polarity_score_list

[{'neg': 0.0,
  'neu': 0.323,
  'pos': 0.677,
  'compound': 0.8874,
  'label': 'nan very kind and caring  provide comfortable seats'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'label': 'nan nan nan'},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0, 'label': 'nan nan nan'},
 {'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'compound': 0.0,
  'label': 'nan nan answer all my questions'},
 {'neg': 0.0,
  'neu': 0.831,
  'pos': 0.169,
  'compound': 0.9042,
  'label': 'i like the facility and the doctor   i felt comfortable and felt like the doctor took time with me  greeting me   discussing all the issues i wanted to    i didn     t feel rushed    i had my appt  and med within a hour  make sure the customer knows where you called the prescription into   i     ve moved a lot and had several pharmacy     s    i verified before i left the appt '},
 {'neg': 0.0,
  'neu': 0.748,
  'pos': 0.252,
  'compound': 0.659,
  'label': 'nan very caring staff   plenty of time for me to ask q

In [9]:
# Creating a new dataframe to showing the new data side by side with the previous one
# We are only interested in the compunt score (aggregated score)
reason_score_reviews['polarity_score'] = pd.DataFrame(reason_score_polarity_score_list)['compound']
reason_score_reviews['positive_score'] = pd.DataFrame(reason_score_polarity_score_list)['pos']
reason_score_reviews['negative_score'] = pd.DataFrame(reason_score_polarity_score_list)['neu']

reason_score_reviews[['polarity_score']] = reason_score_reviews[['polarity_score']].fillna(0)
reason_score_reviews[['positive_score']] = reason_score_reviews[['positive_score']].fillna(0)
reason_score_reviews[['negative_score']] = reason_score_reviews[['negative_score']].fillna(0)

In [11]:
reason_score_reviews['polarity_score'].head()

0    0.8874
1    0.0000
2    0.0000
3    0.0000
4    0.9042
Name: polarity_score, dtype: float64

In [12]:
data['polarity_score'] = reason_score_reviews['polarity_score']

In [13]:
data.to_csv('data_with_polarity.csv',index=False)

In [10]:
reason_score_reviews.head()

REVIEW_ID           MEMBER_ID                        NPSCLINIC__C  \
0  a2p1U000000RoK0QAK  0011U00000riNolQAE        Arvada City of Arvada Clinic   
1  a2p3g000000LAe2AAG  0011U00000riuzfQAA  Seven Hills Lombardo Center Clinic   
2  a2p3g000000LAenAAG  0011U00000rieIYQAY           Monroe Campus Park Clinic   
3  a2p1U000000l8FEQAY  0011U00000rjZH9QAM       Akron-White Pond Drive Clinic   
4  a2p1U000000Rv9VQAS  0011U00000rjesHQAQ      Euclid Lincoln Electric Clinic   

  APPOINTMENTPROVIDER__C SURVEYDATE__C SURVEY_MONTH SURVEY_YEAR  \
0                    NaN    07-09-2018       Jul-18        2018   
1     0031U00000olN5ZQAU    07-01-2020       Jul-20        2020   
2     0031U00000olN7aQAE     6/27/2020       Jun-20        2020   
3     0031U00000olN71QAE    10/25/2019       Oct-19        2019   
4     0031U00000olN8YQAU     9/24/2019       Sep-19        2019   

   SURVEYNUMBER__C  NPS                                  REASONNPSSCORE__C  \
0            20536   10                                                NaN   
1      11752005028    5                                                NaN   
2      11737091171   10                                                NaN   
3            31790   10                                                NaN   
4            31283    8  I like the facility and the doctor.  I felt co...   

   ... positive_score negative_score sentiment sentiment_label  nps_label  \
0  ...          0.677          0.323         1        Positive   Promoter   
1  ...          0.000          1.000         0         Neutral  Detractor   
2  ...          0.000          1.000         0         Neutral   Promoter   
3  ...          0.000          1.000         0         Neutral   Promoter   
4  ...          0.169          0.831         1        Positive    Passive   

                          CLIENT NAME                  PARENT CLIENT NAME  \
0                   State of Colorado                   State of Colorado   
1  Cleveland Bakers & Teamsters (CBT)  Cleveland Bakers & Teamsters (CBT)   
2                        Union County                        Union County   
3  Cleveland Bakers & Teamsters (CBT)  Cleveland Bakers & Teamsters (CBT)   
4                    Lincoln Electric                    Lincoln Electric   

     PARENT_CLIENT_ID MEMBER_DOB MEMBER_STREET  
0  0011U00000rhpLpQAI        NaN           NaN  
1  0011U00000rhpN6QAI        NaN           NaN  
2  0011U00000rhpLjQAI        NaN           NaN  
3  0011U00000rhpN6QAI        NaN           NaN  
4  0011U00000rhpMwQAI        NaN           NaN  

[5 rows x 108 columns]

In [78]:
# Creating a function to classify sentiment scores based on the polarity score
def sentiment_label(row):
    if row['polarity_score'] > 0 :
        sentiment_score = "Positive" # Positive sentiment
    if row['polarity_score'] < 0 :
        sentiment_score = "Negative" # Negative sentiment
    if row['polarity_score'] == 0:
        sentiment_score = "Neutral" # Neutral sentiment
    return sentiment_score

def sentiment(row):
    if row['polarity_score'] > 0 :
        sentiment_score = 1 # Positive sentiment
    if row['polarity_score'] < 0 :
        sentiment_score = -1 # Negative sentiment
    if row['polarity_score'] == 0:
        sentiment_score = 0 # Neutral sentiment
    return sentiment_score

In [79]:
# Creating a new column which will contain sentiment scores based on the polarity score of each feedback
reason_score_reviews['sentiment'] = reason_score_reviews.apply(sentiment, axis=1)
reason_score_reviews['sentiment_label'] = reason_score_reviews.apply(sentiment_label, axis=1)


In [80]:
# Fixing polarity scores and related labels
reason_score_reviews.loc[reason_score_reviews['reviews'].str.match('^(?=.*rude)|(?=.*worst)|(?=.*frustrated)|(?=.*angry)|(?=.*terrible)|(?=.*disrespectful)|(?=.*irresponsibly)|(?=.*irresponsible)|(?=.*improper)|(?=.*wasted)|(?=.*violate)|(?=.*violations)|(?=.*aren t worth)|(?=.*undress)|(?=.*didnt treat me)|(?=.*horrible)|(?=kill)|(?=.*danger)|(?=.*dangerous)|(?=rough)|(?=.*dismissed)|(?=.*upsetting)|(?=.*out of line)|(?=.*uncomfortable)|(?=.*can t breathe)|(?=.*breast)|(?=.*was misdiagnosed)|(?=.*did not examine me)|(?=.*forced to participate)|(?=.*forced into it)'), 'sentiment'] = 2
reason_score_reviews.loc[reason_score_reviews['reviews'].str.match('^(?=.*rude)|(?=.*worst)|(?=.*frustrated)|(?=.*angry)|(?=.*terrible)|(?=.*disrespectful)|(?=.*irresponsibly)|(?=.*irresponsible)|(?=.*improper)|(?=.*wasted)|(?=.*violate)|(?=.*violations)|(?=.*aren t worth)|(?=.*undress)|(?=.*didnt treat me)|(?=.*horrible)|(?=kill)|(?=.*danger)|(?=.*dangerous)|(?=rough)|(?=.*dismissed)|(?=.*upsetting)|(?=.*out of line)|(?=.*uncomfortable)|(?=.*can t breathe)|(?=.*breast)|(?=.*was misdiagnosed)|(?=.*did not examine me)|(?=.*forced to participate)|(?=.*forced into it)'), 'sentiment_label'] = 'Extreme'
reason_score_reviews.loc[(reason_score_reviews['polarity_score']>0) & (reason_score_reviews['sentiment_label']=='Extreme'), 'polarity_score']*=-1

reason_score_reviews.loc[(reason_score_reviews['NPS']>=7) & (reason_score_reviews['sentiment_label']=='Negative'), 'sentiment']=1
reason_score_reviews.loc[(reason_score_reviews['NPS']>=7) & (reason_score_reviews['sentiment_label']=='Negative'), 'sentiment_label']='Positive'
reason_score_reviews.loc[(reason_score_reviews['NPS']>=7) & (reason_score_reviews['polarity_score']<0), 'polarity_score']*=-1

reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*good)|(?=.*great)|(?=.*awesome)|(?=.*very much)|(?=.*friendly)|(?=.*appreciate)|(?=.*compassionate)|(?=.*went well)|(?=.*very thorough)|(?=.*assisted)|(?=.*easy)|(?=.*enjoyed)|(?=.*i like)|(?=.*i liked)|(?=.*nice)|(?=.*love)|(?=.*caring)|(?=.*impressed)|(?=.*knowledgeable)|(?=.*very comfortable)|(?=.*very professional)|(?=.*achieved)|(?=.*courteous)|(?=.*listen to what i had to say)|(?=.*right amount of time)|(?=.*on time)|(?=.*personable)|(?=.*efficient)|(?=.*fantastic)|(?=.*punctual)|(?=.*go over issues)|(?=.*got me in same day)|(?=.*all my concerns)|(?=.*everything)|(?=.*sat and talked)') & (reason_score_reviews['NPS']>=5), 'sentiment'] = 1
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*good)|(?=.*great)|(?=.*awesome)|(?=.*very much)|(?=.*friendly)|(?=.*appreciate)|(?=.*compassionate)|(?=.*went well)|(?=.*very thorough)|(?=.*assisted)|(?=.*easy)|(?=.*enjoyed)|(?=.*i like)|(?=.*i liked)|(?=.*nice)|(?=.*love)|(?=.*caring)|(?=.*impressed)|(?=.*knowledgeable)|(?=.*very comfortable)|(?=.*very professional)|(?=.*achieved)|(?=.*courteous)|(?=.*listen to what i had to say)|(?=.*right amount of time)|(?=.*on time)|(?=.*personable)|(?=.*efficient)|(?=.*fantastic)|(?=.*punctual)|(?=.*go over issues)|(?=.*got me in same day)|(?=.*all my concerns)|(?=.*everything)|(?=.*sat and talked)') & (reason_score_reviews['NPS']>=5), 'sentiment_label'] = 'Positive'
reason_score_reviews.loc[(reason_score_reviews['polarity_score']<0) & (reason_score_reviews['sentiment_label']=='Positive'), 'polarity_score']*=-1

In [82]:
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*wasn t treated)|(?=.*waited)|(?=.*not as caring)|(?=.*did not think)|(?=.*didn t think)|(?=.*did not get)|(?=.*didn t get)|(?=.*felt rushed)|(?=.*was rushed)|(?=.*seemed rushed)|(?=.*did not fix)|(?=.*didn t fix)|(?=.*didn t listen)|(?=.*did not listen)|(?=.*ignored)|(?=.*unprofessional)|(?=.*don t think)|(?=.*do not think)|(?=.*waiting)|(?=.*talked too much)|(?=.*please)|(?=.*took longer)|(?=.*didn t show)|(?=.*did not show)|(?=.*never)|(?=.*difficult)|(?=.*unable)|(?=.*didn t look)|(?=.*did not look)|(?=.*not being treated)|(?=.*was not)|(?=.*confused)|(?=.*did not recognize)|(?=.*didn t recognize)|(?=.*do not like)|(?=.*don t like)|(?=.*wasn t greeted)|(?=.*was not greeted)|(?=.*need to improve)|(?=.*no one)|(?=.*lacking)|(?=.*no solution)|(?=.*didn t understand)|(?=.*did not understand)|(?=.*unorganized)|(?=.*my issues)|(?=.*without)|(?=.*no follow up)|(?=.*doesn t seem)|(?=.*does not seem)|(?=.*would not call)|(?=.*wouldn t call)|(?=.*were not)|(?=.*not very)|(?=.*not answered)|(?=.*no answer)|(?=.*conflict)|(?=.*didnt treat)|(?=.*didn t treat)|(?=.*did not treat)|(?=.*lack)|(?=.*lacking)|(?=.*lacked)|(?=.*need to)|(?=.*not near as caring)|(?=.*already expressed)|(?=.*far away)') & (reason_score_reviews['NPS']<=4), 'sentiment'] = -1
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*wasn t treated)|(?=.*waited)|(?=.*not as caring)|(?=.*did not think)|(?=.*didn t think)|(?=.*did not get)|(?=.*didn t get)|(?=.*felt rushed)|(?=.*was rushed)|(?=.*seemed rushed)|(?=.*did not fix)|(?=.*didn t fix)|(?=.*didn t listen)|(?=.*did not listen)|(?=.*ignored)|(?=.*unprofessional)|(?=.*don t think)|(?=.*do not think)|(?=.*waiting)|(?=.*talked too much)|(?=.*please)|(?=.*took longer)|(?=.*didn t show)|(?=.*did not show)|(?=.*never)|(?=.*difficult)|(?=.*unable)|(?=.*didn t look)|(?=.*did not look)|(?=.*not being treated)|(?=.*was not)|(?=.*confused)|(?=.*did not recognize)|(?=.*didn t recognize)|(?=.*do not like)|(?=.*don t like)|(?=.*wasn t greeted)|(?=.*was not greeted)|(?=.*need to improve)|(?=.*no one)|(?=.*lacking)|(?=.*no solution)|(?=.*didn t understand)|(?=.*did not understand)|(?=.*unorganized)|(?=.*my issues)|(?=.*without)|(?=.*no follow up)|(?=.*doesn t seem)|(?=.*does not seem)|(?=.*would not call)|(?=.*wouldn t call)|(?=.*were not)|(?=.*not very)|(?=.*not answered)|(?=.*no answer)|(?=.*conflict)|(?=.*didnt treat)|(?=.*didn t treat)|(?=.*did not treat)|(?=.*lack)|(?=.*lacking)|(?=.*lacked)|(?=.*need to)|(?=.*not near as caring)|(?=.*already expressed)|(?=.*far away)') & (reason_score_reviews['NPS']<=4), 'sentiment_label'] = 'Negative'
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*wasn t treated)|(?=.*waited)|(?=.*not as caring)|(?=.*did not think)|(?=.*didn t think)|(?=.*did not get)|(?=.*didn t get)|(?=.*felt rushed)|(?=.*was rushed)|(?=.*seemed rushed)|(?=.*did not fix)|(?=.*didn t fix)|(?=.*didn t listen)|(?=.*did not listen)|(?=.*ignored)|(?=.*unprofessional)|(?=.*don t think)|(?=.*do not think)|(?=.*waiting)|(?=.*talked too much)|(?=.*please)|(?=.*took longer)|(?=.*didn t show)|(?=.*did not show)|(?=.*never)|(?=.*difficult)|(?=.*unable)|(?=.*didn t look)|(?=.*did not look)|(?=.*not being treated)|(?=.*was not)|(?=.*confused)|(?=.*did not recognize)|(?=.*didn t recognize)|(?=.*do not like)|(?=.*don t like)|(?=.*wasn t greeted)|(?=.*was not greeted)|(?=.*need to improve)|(?=.*no one)|(?=.*lacking)|(?=.*no solution)|(?=.*didn t understand)|(?=.*did not understand)|(?=.*unorganized)|(?=.*my issues)|(?=.*without)|(?=.*no follow up)|(?=.*doesn t seem)|(?=.*does not seem)|(?=.*would not call)|(?=.*wouldn t call)|(?=.*were not)|(?=.*not very)|(?=.*not answered)|(?=.*no answer)|(?=.*conflict)|(?=.*didnt treat)|(?=.*didn t treat)|(?=.*did not treat)|(?=.*lack)|(?=.*lacking)|(?=.*lacked)|(?=.*need to)|(?=.*not near as caring)|(?=.*already expressed)|(?=.*far away)|(?=.*dirty)|(?=.*don t have resources)|(?=.*dont lie)|(?=.*too long)|(?=.*forgot)|(?=.*give proper)|(?=.*not treating)|(?=.*worthless)|(?=.*doesn t look well)|(?=.*don t get)|(?=.*way to long)|(?=.*not willing)|(?=.*no examination)|(?=.*failed)|(?=.*didn t follow up)|(?=.*doesn t follow up)|(?=.*was cold)|(?=.*little compassion)|(?=.*inexperienced)|(?=.*didn t even)|(?=.*needs to be cleaned)|(?=.*forgot to call)|(?=.*did not hear)|(?=.*superficial)|(?=.*afraid to touch)|(?=.*did not trust)|(?=.*far too long)|(?=.*don t have any resources)|(?=.*not happy)|(?=.*made me wait)|(?=.*awful)|(?=.*dirty)|(?=.*nobody grteeted)|(?=.*won t call)|(?=.*did not work)|(?=.*entire problem was)|(?=.*didn t even acknowledge)|(?=.*doesn t care)|(?=.*incompetent)|(?=.*not knowledgeable)|(?=.*didn t make appointment right)|(?=.*should not have)|(?=.*had to wait awhile)|(?=.*way too long)|(?=.*frustrating)|(?=.*they don t do)|(?=.*than the person you have hired)|(?=.*wasn t very clean)|(?=.*made me very sick)|(?=.*we were told)|(?=.*what is going on)|(?=.*don t recommend)|(?=.*only thing they cared about)|(?=.*did not seem to care)|(?=.*pretty disappointed)|(?=.*mess up my medicine)|(?=.*very demeaning)|(?=.*unfriendly)|(?=.*very disappointed)|(?=.*didn t have any good)|(?=.*wasn t much true interest)|(?=.*not acceptable)|(?=.*does not listen)|(?=.*did not listen)|(?=.*would not refill)|(?=.*not happening)|(?=.*not great)|(?=.*new very little)|(?=.*to long of a time)|(?=.*poor instructions)|(?=.*poor service)|(?=.*was charged an excessive amount)|(?=.*thrilled i was able)'), 'sentiment'] = -1
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*wasn t treated)|(?=.*waited)|(?=.*not as caring)|(?=.*did not think)|(?=.*didn t think)|(?=.*did not get)|(?=.*didn t get)|(?=.*felt rushed)|(?=.*was rushed)|(?=.*seemed rushed)|(?=.*did not fix)|(?=.*didn t fix)|(?=.*didn t listen)|(?=.*did not listen)|(?=.*ignored)|(?=.*unprofessional)|(?=.*don t think)|(?=.*do not think)|(?=.*waiting)|(?=.*talked too much)|(?=.*please)|(?=.*took longer)|(?=.*didn t show)|(?=.*did not show)|(?=.*never)|(?=.*difficult)|(?=.*unable)|(?=.*didn t look)|(?=.*did not look)|(?=.*not being treated)|(?=.*was not)|(?=.*confused)|(?=.*did not recognize)|(?=.*didn t recognize)|(?=.*do not like)|(?=.*don t like)|(?=.*wasn t greeted)|(?=.*was not greeted)|(?=.*need to improve)|(?=.*no one)|(?=.*lacking)|(?=.*no solution)|(?=.*didn t understand)|(?=.*did not understand)|(?=.*unorganized)|(?=.*my issues)|(?=.*without)|(?=.*no follow up)|(?=.*doesn t seem)|(?=.*does not seem)|(?=.*would not call)|(?=.*wouldn t call)|(?=.*were not)|(?=.*not very)|(?=.*not answered)|(?=.*no answer)|(?=.*conflict)|(?=.*didnt treat)|(?=.*didn t treat)|(?=.*did not treat)|(?=.*lack)|(?=.*lacking)|(?=.*lacked)|(?=.*need to)|(?=.*not near as caring)|(?=.*already expressed)|(?=.*far away)|(?=.*dirty)|(?=.*don t have resources)|(?=.*dont lie)|(?=.*too long)|(?=.*forgot)|(?=.*give proper)|(?=.*not treating)|(?=.*worthless)|(?=.*doesn t look well)|(?=.*don t get)|(?=.*way to long)|(?=.*not willing)|(?=.*no examination)|(?=.*failed)|(?=.*didn t follow up)|(?=.*doesn t follow up)|(?=.*was cold)|(?=.*little compassion)|(?=.*inexperienced)|(?=.*didn t even)|(?=.*needs to be cleaned)|(?=.*forgot to call)|(?=.*did not hear)|(?=.*superficial)|(?=.*afraid to touch)|(?=.*did not trust)|(?=.*far too long)|(?=.*don t have any resources)|(?=.*not happy)|(?=.*made me wait)|(?=.*awful)|(?=.*dirty)|(?=.*nobody grteeted)|(?=.*won t call)|(?=.*did not work)|(?=.*entire problem was)|(?=.*didn t even acknowledge)|(?=.*doesn t care)|(?=.*incompetent)|(?=.*not knowledgeable)|(?=.*didn t make appointment right)|(?=.*should not have)|(?=.*had to wait awhile)|(?=.*way too long)|(?=.*frustrating)|(?=.*they don t do)|(?=.*than the person you have hired)|(?=.*wasn t very clean)|(?=.*made me very sick)|(?=.*we were told)|(?=.*what is going on)|(?=.*don t recommend)|(?=.*only thing they cared about)|(?=.*did not seem to care)|(?=.*pretty disappointed)|(?=.*mess up my medicine)|(?=.*very demeaning)|(?=.*unfriendly)|(?=.*very disappointed)|(?=.*didn t have any good)|(?=.*wasn t much true interest)|(?=.*not acceptable)|(?=.*does not listen)|(?=.*did not listen)|(?=.*would not refill)|(?=.*not happening)|(?=.*not great)|(?=.*new very little)|(?=.*to long of a time)|(?=.*poor instructions)|(?=.*poor service)|(?=.*was charged an excessive amount)|(?=.*thrilled i was able)'), 'sentiment_label'] = 'Negative'
reason_score_reviews.loc[(reason_score_reviews['polarity_score']>0) & (reason_score_reviews['sentiment_label']=='Negative'), 'polarity_score']*=-1

reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*amazing)|(?=.*can t think of anything)|(?=.*completely satisfied)|(?=.*no complaints)|(?=.*perfect)|(?=.*was fine)|(?=.*was nice)|(?=.*very happy)|(?=.*well communication)|(?=.*enough time with me)|(?=.*not a single thing)|(?=.*couldn t ask for anything more)|(?=.*very satisfied)|(?=.*nothing to say)|(?=.*nothing to report)|(?=.*very pleasant)|(?=.*was well)|(?=.*completely happy)|(?=.*was pleased)|(?=.*all employees use)|(?=.*super nice)|(?=.*never had a doctor spend)|(?=.*very pleased)|(?=.*it was okay)|(?=.*was quick)|(?=.*convenient high quality)|(?=.*very welcoming)|(?=.*provide great care)|(?=.*very nice)|(?=.*treat you like family)|(?=.*always on time)|(?=.*very helpful)|(?=.*i am fortunate)|(?=.*best coarse of action)|(?=.*very convenient)|(?=.*i iike the doctor)|(?=.*great service)|(?=.*wasn t too long)|(?=.*extremely convenient)|(?=.*service is good)|(?=.*i like being able to)|(?=.*overall i liked)|(?=.*paladinais the city s health)'), 'sentiment'] = 1
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=.*amazing)|(?=.*can t think of anything)|(?=.*completely satisfied)|(?=.*no complaints)|(?=.*perfect)|(?=.*was fine)|(?=.*was nice)|(?=.*very happy)|(?=.*well communication)|(?=.*enough time with me)|(?=.*not a single thing)|(?=.*couldn t ask for anything more)|(?=.*very satisfied)|(?=.*nothing to say)|(?=.*nothing to report)|(?=.*very pleasant)|(?=.*was well)|(?=.*completely happy)|(?=.*was pleased)|(?=.*all employees use)|(?=.*super nice)|(?=.*never had a doctor spend)|(?=.*very pleased)|(?=.*it was okay)|(?=.*was quick)|(?=.*convenient high quality)|(?=.*very welcoming)|(?=.*provide great care)|(?=.*very nice)|(?=.*treat you like family)|(?=.*always on time)|(?=.*very helpful)|(?=.*i am fortunate)|(?=.*best coarse of action)|(?=.*very convenient)|(?=.*i iike the doctor)|(?=.*great service)|(?=.*wasn t too long)|(?=.*extremely convenient)|(?=.*service is good)|(?=.*i like being able to)|(?=.*overall i liked)|(?=.*paladinais the city s health)'), 'sentiment_label'] = 'Positive'
reason_score_reviews.loc[reason_score_reviews['reviews'].str.contains('^(?=convenient)') & (reason_score_reviews['NPS']==6), 'sentiment_label'] = 'Positive'
reason_score_reviews.loc[(reason_score_reviews['polarity_score']<0) & (reason_score_reviews['sentiment_label']=='Positive'), 'polarity_score']*=-1

reason_score_reviews.loc[(reason_score_reviews['NPS']>=9) & (reason_score_reviews['NPS']<=10), 'nps_label'] = 'Promoter'
reason_score_reviews.loc[(reason_score_reviews['NPS']>=7) & (reason_score_reviews['NPS']<=8), 'nps_label'] = 'Passive'
reason_score_reviews.loc[reason_score_reviews['NPS']<=6, 'nps_label'] = 'Detractor'


Promoter     21421
Passive       3068
Detractor     1411
Name: nps_label, dtype: int64

In [114]:
#reason_score_reviews['nps_label'].value_counts()

In [115]:
#reason_score_reviews["sentiment_label"].value_counts()

In [108]:
client_df = pd.read_csv('Client Master Clean Data_Client Master List_clean.csv')

In [109]:
c_df = client_df[['CLIENT NAME', 'PARENT CLIENT NAME', 'CLIENT_ID','PARENT_CLIENT_ID']]

In [110]:
c_df = c_df.drop_duplicates(subset = ['CLIENT_ID'],keep=False)

In [111]:
after_client_join = reason_score_reviews.merge(c_df, on='CLIENT_ID', how='left')

In [113]:
after_client_join.to_csv('final_2018_2020.csv',index = False)